In [1]:

import random as rd
import copy
from itertools import combinations
import numpy as np
from os import listdir
from os.path import isfile, join
from os import path
import argparse
import csv
import time
from pathlib import Path


In [2]:
import numpy as np

__all__ = ['QAData', 'QAReader', 'init_solution']


def init_solution(n):
    return np.random.permutation(np.arange(n))


class QAData:
    def __init__(self, n, distances, flows):
        self.n = n
        self.distances = distances
        self.flows = flows

    def compute_cost(self, solution, **kwargs):
        cost = 0
        for i in range(self.n):
            for j in range(self.n):
                dist = self.distances[solution[i]][solution[j]]
                flow = self.flows[i][j]
                cost += flow * dist

        if 'method' not in kwargs.keys():
            return cost
        elif kwargs['method'] == 'guided':
            augmented_part = 0
            mu = kwargs['mu']
            indicator_func = kwargs['indicator']
            penalty = kwargs['penalty']
            for u in range(self.n):
                for v in range(self.n):
                    augmented_part += indicator_func[u][v] * penalty[u][v]
            return cost + mu * augmented_part


class QAReader:
    def __init__(self):
        pass

    def __call__(self, path):
        with open(path, "r") as f:
            n = int(f.readline().strip())
            distances, flows = np.empty((n, n)), np.empty((n, n))
            for i in range(n):
                flows[i] = (list(map(int, f.readline().split())))
            _ = f.readline()
            for j in range(n):
                distances[j] = (list(map(int, f.readline().split())))
        return QAData(n, distances, flows)

In [3]:
class IteratedLocalSearch:
    def __init__(self, data, method, verbose, n_iter):
        self.data = data
        self.verbose = verbose
        self.method = method
        self.iter_amount = n_iter
        self.solution = init_solution(self.data.n)
        self.current_cost = self.data.compute_cost(self.solution)
        self.solver = LocalSearch(self.data, method, False, int(self.iter_amount * 0.1))
        self.cost_history = []

    def perturbation(self):
        k = rd.randint(2, self.data.n)
        indexes = np.random.choice(np.arange(self.data.n), k, replace=False)
        shuffled_indexes = np.random.permutation(indexes)
        new_solution = self.solution.copy()
        new_solution[indexes] = self.solution[shuffled_indexes]
        return new_solution

    def acceptance_criterion(self, new_solution):
        if self.data.compute_cost(self.solution) > self.data.compute_cost(new_solution):
            self.solution = new_solution

    def __call__(self):
        if self.verbose:
            print(f'Starting value of cost func is {self.data.compute_cost(self.solution)}')
        self.solution, cost = self.solver(self.solution)
        for _ in range(self.iter_amount):
            new_solution, cost = self.solver(self.perturbation())
            self.cost_history.append(cost)
            self.acceptance_criterion(new_solution)
        final_cost = self.data.compute_cost(self.solution)
        if self.verbose:
            print('Final cost {}'.format(final_cost))
        return self.solution, final_cost

In [4]:
class LocalSearch:
    def __init__(self, data, method, verbose, n_iter, solution=None):
        self.data = data
        self.method = method
        self.verbose = verbose
        self.iter_amount = n_iter
        self.solution = init_solution(self.data.n)
        self.current_cost = self.data.compute_cost(self.solution)
        self.patience = int(0.2 * self.iter_amount)
        self.improved = True
        self.no_improvements = 0


    def count_delta(self, r, s):
        diff = 0
        pi = self.solution
        for k in range(self.data.n):
            if k != r and k != s:
                diff += (self.data.flows[k, r] + self.data.flows[r, k]) * \
                        (self.data.distances[pi[s], pi[k]] - self.data.distances[pi[r], pi[k]]) + \
                        (self.data.flows[k, s] + self.data.flows[s, k]) * \
                        (self.data.distances[pi[r], pi[k]] - self.data.distances[pi[s], pi[k]])
        return diff

    def count_delta_with_previous(self, previous, u, v, r, s):
        pi = self.solution
        return previous + 2 * (self.data.flows[r, u] - self.data.flows[r, v] + self.data.flows[s, v] - self.data.flows[s, u]) * \
                          (self.data.distances[pi[s], pi[u]] - self.data.distances[pi[s], pi[v]] + self.data.distances[pi[r], pi[v]] - self.data.distances[pi[r], pi[u]])


    def best_improvement(self):
        if self.verbose:
            print(f'Starting value of cost func is {self.data.compute_cost(self.solution)}')
        previous_opt = None
        previous_delta = None
        for _ in range(self.iter_amount):
            comb = list(combinations(np.arange(self.data.n, dtype=np.int32), 2))
            if not self.improved:
                self.no_improvements += 1
            if self.no_improvements == self.patience:
                break
            self.improved = False
            min_delta = previous_delta if previous_delta is not None else 0
            optimal_opt = None
            for opt in comb:
                opt = list(opt)
                if previous_opt is not None and not opt == previous_opt:
                    delta = self.count_delta_with_previous(previous_delta, *opt, *previous_opt)
                elif opt == previous_opt:
                    continue
                else:
                    delta = self.count_delta(*opt)
                if delta < min_delta:
                    min_delta = delta
                    optimal_opt = opt
            if optimal_opt is not None:
                self.solution[optimal_opt] = self.solution[optimal_opt][::-1]
                self.improved = True
                previous_opt = optimal_opt
                previous_delta = min_delta
            else:
                self.improved = False
        final_cost = self.data.compute_cost(self.solution)
        if self.verbose:
            print('Final cost {}'.format(final_cost))
        return self.solution, final_cost

    def __call__(self, solution=None, **kwargs):
            return self.best_improvement()

In [5]:
best_known = {
    'tai20a': 703482,
    'tai40a': 3139370,
    'tai60a': 7205962,
    'tai80a': 13499184,
    'tai100a': 21044752,
}

In [9]:
            n_iter = 100
            file_res = open("C:\\Users\\Vlad\\Desktop\\Lab4\\result.csv", 'w')
            columns_names = ['File name', 'Method', 'Best known', 'Result', 'Time']
            writer = csv.DictWriter(file_res, fieldnames=columns_names)
            writer.writeheader()
            reader = QAReader()
            benchmarks = [f for f in listdir('C:\\Users\\Vlad\\Desktop\\Lab4\\Data') if isfile(join('C:\\Users\\Vlad\\Desktop\\Lab4\\Data', f))]
            for file in benchmarks:
                data = reader(path.join('C:\\Users\\Vlad\\Desktop\\Lab4\\Data', file))
                algorithms = [(name, f(data, 'best_improvement', True, n_iter)) for name, f in [("local search", LocalSearch), ("iterated local search", IteratedLocalSearch)] if callable(f)]
                for name, algorithm in algorithms:
                    Path(f"./{name}").mkdir(parents=True, exist_ok=True)
                    print(f'{name} working on {file}')
                    start_time = time.time()
                    solution, final_cost = algorithm()
                    work_time = round(time.time() - start_time, 4)
                    np.savetxt(f'./{name}/{file}.sol', solution.reshape(1, solution.shape[0]), fmt='%d')
                    writer.writerow({
                        'File name': file,
                        'Method': name,
                        'Best known': best_known[file],
                        'Result': final_cost,
                        'Time': work_time
                    })

local search working on tai100a
Starting value of cost func is 24028618.0
Final cost 23632866.0
iterated local search working on tai100a
Starting value of cost func is 24131030.0
Final cost 23753276.0
local search working on tai20a
Starting value of cost func is 890416.0
Final cost 869872.0
iterated local search working on tai20a
Starting value of cost func is 882628.0
Final cost 847474.0
local search working on tai40a
Starting value of cost func is 3795200.0
Final cost 3683642.0
iterated local search working on tai40a
Starting value of cost func is 3692314.0
Final cost 3584814.0
local search working on tai60a
Starting value of cost func is 8478988.0
Final cost 8324726.0
iterated local search working on tai60a
Starting value of cost func is 8482300.0
Final cost 8151176.0
local search working on tai80a
Starting value of cost func is 15480946.0
Final cost 15245826.0
iterated local search working on tai80a
Starting value of cost func is 15669124.0
Final cost 15047198.0


In [10]:
print('Local Search')
for file in listdir('C:\\Users\\Vlad\\local search'):
    sol = open(f'C:\\Users\\Vlad\\local search\\{file}', 'r').read()
    print(f'Name: {file}. Solution {sol}')

Local Search
Name: tai100a.sol. Solution 93 45 12 55 42 31 80 86 22 68 24 10 25 99 81 30 91 29 8 32 37 69 76 70 58 87 47 95 57 4 17 35 39 38 41 60 46 54 67 85 18 23 62 26 0 61 21 15 88 19 16 7 50 43 77 97 6 72 11 34 44 64 71 49 96 52 1 14 98 94 40 59 79 13 84 27 89 53 65 20 83 78 56 36 82 75 66 92 51 9 74 73 3 90 2 5 33 48 28 63

Name: tai20a.sol. Solution 17 10 19 15 0 4 7 5 2 13 12 1 14 18 9 8 11 3 6 16

Name: tai40a.sol. Solution 16 33 9 7 35 2 0 19 3 22 18 21 30 25 27 1 39 24 28 15 34 17 14 36 29 26 37 10 13 31 11 32 6 12 23 38 8 5 20 4

Name: tai60a.sol. Solution 38 23 32 40 53 35 45 29 31 26 56 58 43 24 16 30 3 41 19 59 10 47 11 50 37 52 27 12 44 15 57 4 21 39 7 17 51 25 33 1 34 13 14 46 55 6 5 28 36 0 20 9 22 54 42 2 18 8 48 49

Name: tai80a.sol. Solution 39 60 5 74 54 69 38 9 42 53 46 66 78 58 6 18 67 40 8 43 51 7 49 36 10 21 23 77 13 52 55 65 41 16 37 27 63 76 59 17 62 14 56 20 25 47 28 48 12 79 19 33 61 3 26 35 4 22 50 15 11 31 0 32 24 71 75 68 73 30 45 1 57 72 70 2 34 44 64 

In [11]:
print('Iterated Local Search')
for file in listdir('C:\\Users\\Vlad\\iterated local search'):
    sol = open(f'C:\\Users\\Vlad\\iterated local search\\{file}', 'r').read()
    print(f'Name: {file}. Solution {sol}')

Iterated Local Search
Name: tai100a.sol. Solution 64 87 81 44 74 45 79 40 29 54 18 77 75 6 11 16 32 71 67 49 39 23 78 4 27 73 55 20 69 57 28 99 15 62 91 85 13 37 12 7 53 34 17 86 46 33 51 97 47 43 68 61 1 9 60 36 3 76 41 31 95 35 88 42 66 94 82 25 52 19 10 98 70 48 96 83 58 93 65 38 8 50 5 22 0 92 80 90 89 59 63 56 30 2 14 21 24 84 72 26

Name: tai20a.sol. Solution 6 14 1 2 8 18 10 5 7 16 15 19 11 0 17 13 9 3 12 4

Name: tai40a.sol. Solution 10 26 32 20 17 24 13 9 16 1 28 34 19 15 39 35 4 18 27 37 14 22 29 2 3 0 31 8 11 12 23 38 25 7 5 30 33 36 6 21

Name: tai60a.sol. Solution 52 10 17 41 44 33 5 39 50 29 0 59 42 25 47 48 6 9 51 34 22 38 13 26 18 49 16 31 2 30 40 24 45 21 32 23 15 8 12 28 56 11 27 1 7 37 57 4 35 55 46 3 43 20 58 36 54 14 53 19

Name: tai80a.sol. Solution 60 79 71 35 70 21 29 53 50 39 49 23 22 72 14 31 43 69 34 18 38 73 52 64 44 57 24 62 8 68 13 19 0 12 6 5 16 1 7 74 55 33 37 3 51 59 30 78 40 27 75 15 9 76 66 61 25 36 2 58 11 32 48 42 17 10 45 77 20 63 46 65 28 47 26 41